<a href="https://colab.research.google.com/github/polinliv/livanou/blob/main/%CE%91%CE%BD%CF%84%CE%AF%CE%B3%CF%81%CE%B1%CF%86%CE%BF_greedy_motif_search_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**GreedyMotifSearch**

Based on code from:
1. Compeau & Pevzner, Bioinformatics Algorithms
2. https://github.com/rivers-yao/Coursework/blob/master/Bioinformatics%20Algorithm/Week2

In [ ]:
import math

In [ ]:
# Auxialiary Functions
def SymbolToNumber(Symbol):
    """
    Converts base to number (in lexicograpical order)

    Symbol: the letter to be converted (str)

    Returns: the number corresponding to that base (int)
    """
    if Symbol == "A":
        return 0
    elif Symbol == "C":
        return 1
    elif Symbol == "G":
        return 2
    elif Symbol == "T":
        return 3
    else:
        print(f'Unexpected symbol {Symbol} in function SymbolToNumber()')
        return None


def NumberToSymbol(index):
    """
    Finds base from number (in lexicographical order)

    index: the number to be converted (int)

    Returns: the base corresponding to index (str)
    """
    if index == 0:
        return str("A")
    elif index == 1:
        return str("C")
    elif index == 2:
        return str("G")
    elif index == 3:
        return str("T")


def HammingDistance(p, q):
    """
    Finds the number of mismatches between 2 DNA segments of equal lengths

    p: first DNA segment (str)

    q: second DNA segment (str)

    Returns: number of mismatches (int)
    """
    return sum(s1 != s2 for s1, s2 in zip(p, q))


def window(s, k):
    for i in range(1 + len(s) - k):
        yield s[i:i+k]


def ProfileMostProbable(Text, k, Profile):
    """
    Finds a k-mer that was most likely to be generated by profile among
    all k-mers in Text

    Text: given DNA segment (str)

    k: length of pattern (int)

    Profile: a 4x4 matrix (list)

    Returns: profile-most probable k-mer (str)
    """
    letter = [[] for key in range(k)]
    probable = ""
    hamdict = {}
    index = 1
    for a in range(k):
        for j in "ACGT":
            letter[a].append(Profile[j][a])
    for b in range(len(letter)):
        number = max(letter[b])
        probable += str(NumberToSymbol(letter[b].index(number)))
    for c in window(Text, k):
        for x in range(len(c)):
            y = SymbolToNumber(c[x])
            index *= float(letter[x][y])
        hamdict[c] = index
        index = 1
    for pat, ham in hamdict.items():
        if ham == max(hamdict.values()):
            final = pat
            break
    return final


def Count(Motifs):
    """
    Documentation here
    """
    count = {}
    k = len(Motifs[0])
    for symbol in "ACGT":
        count[symbol] = []
        for i in range(k):
            count[symbol].append(0)
    t = len(Motifs)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count


def FindConsensus(motifs):
    """
    Finds a consensus sequence for given list of motifs

    motifs: a list of motif sequences (list)

    Returns: consensus sequence of motifs (str)
    """
    consensus = ""
    for i in range(len(motifs[0])):
        countA, countC, countG, countT = 0, 0, 0, 0
        for motif in motifs:
            if motif[i] == "A":
                countA += 1
            elif motif[i] == "C":
                countC += 1
            elif motif[i] == "G":
                countG += 1
            elif motif[i] == "T":
                countT += 1
        if countA >= max(countC, countG, countT):
            consensus += "A"
        elif countC >= max(countA, countG, countT):
            consensus += "C"
        elif countG >= max(countC, countA, countT):
            consensus += "G"
        elif countT >= max(countC, countG, countA):
            consensus += "T"
    return consensus


def ProfileMatrix(motifs):
    """
    Finds the profile matrix for given list of motifs

    motifs: list of motif sequences (list)

    Returns: the profile matrix for motifs (list)
    """
    Profile = {}
    A, C, G, T = [], [], [], []
    for j in range(len(motifs[0])):
        countA, countC, countG, countT = 0, 0, 0, 0
        for motif in motifs:
            if motif[j] == "A":
                countA += 1
            elif motif[j] == "C":
                countC += 1
            elif motif[j] == "G":
                countG += 1
            elif motif[j] == "T":
                countT += 1
        A.append(countA)
        C.append(countC)
        G.append(countG)
        T.append(countT)
    Profile["A"] = A
    Profile["C"] = C
    Profile["G"] = G
    Profile["T"] = T
    return Profile


def Score(motifs):
    """
    Finds score of motifs relative to the consensus sequence

    motifs: a list of given motifs (list)

    Returns: score of given motifs (int)
    """
    consensus = FindConsensus(motifs)
    score = 0.0000
    for motif in motifs:
        score += HammingDistance(consensus, motif)
    #print(score)
    return round(score, 4)

**Greedy Motif Search Algorithm**

In [ ]:
def GreedyMotifSearch(DNA, k):
    print(f'Greedy search for motifs of length {k}. ')
    print(f'The best solution that is found is returned.')

    t = len(DNA)
    bestMotifs = []
    bestScore = math.inf
    for string in DNA:
        bestMotifs.append(string[:k])
    base = DNA[0]
    for i in window(base, k):
        # Change here. Should start with one element in motifs and build up.
        # As in the line "motifs ← list with only Dna[0](i,k)"
        # newMotifs = []
        newMotifs = [i]
        # Change here to iterate over len(DNA).
        # Should go through "for j from 1 to |Dna| - 1"
        # for j in range(t):
        for j in range(1, len(DNA)):
            # Change here. Should build up motifs and build profile using them.
            # profile = ProfileMatrix([i])
            profile = ProfileMatrix(newMotifs)
            probable = ProfileMostProbable(DNA[j], k, profile)
            newMotifs.append(probable)

        # Change to < rather < = to ensure getting the most recent hit. As referenced in the instructions:
        # If at any step you find more than one Profile-most probable k-mer in a given string, use the one occurring **first**.
        new_score = Score(newMotifs)
        if new_score < bestScore:
        #if Score(newMotifs) <= bestScore:
            bestScore = new_score
            bestMotifs = newMotifs
    return bestMotifs

**Data: DNA Sequences**

In [ ]:
DNA_sequences = ['GTATCTACTCACGTTGCTACCTGCGGTTCCGATTAGGATCAGGTTATAACGTAGTCGCCCTGATATGACTACATCGTTGCAGGCTTGTTACACGTACTAACATATTGGAAAACG',
'TGCCCACTCCCTGATACGCGTTGACCGATCGTGGTGATTTCTTGATCTAAGTCGCCACCCTCTTCAAGGGCTATACATCAACAGAAATTTGTCTGTTTTAGGTCTGTGCCCGCC',
'CTAGACGTAGCGATATGTCGTATGGGACGGGATGTCGCGTGCAAAAATGACCCCTCGGAACCGGCTAAGGCAGCTGTCTATGAGTGGTATTCCTGCTAGGCGTTAACTCGGACT',
'GAGGACCCGGGTACTTGCAGCAAGGCGTGAAAGAAGAAATAATGGTCTATAAAGTGCCCCTGATTCCTGCTCTCAAGGCACCTCGGTGGGGTGCGTTAGGCATAATTTGTTTGT',
'GCCAACACATCCGACGAGAGGTATCAAGCGTACCGCACGTTCACAGGAGATTCCATGGCGACTTAAGTAGATCGTCGTGCACAAAGGCGCCAGACCGATGATACGCGTACAATC',
'ATTCGTCGACGACACCTCCTGCTCATAGTCGCGACCTTAACCTTCTCCACCTTGACTACCCCGGCCTGAGCAAACGCTTCATTAAACATAGAAATTAACCAAGACGGTGGATAG',
'CGCGGCTCTAACTCAAACCCGACAACTCCGAGGATCCCGGGTGAGCATTATCGTGGGCAATTTTACTGCCGGGTTCTGGTAGTAGTCGCGCAGGTAACGCGCGAAGCTTGTGTC',
'TCAATGACAACGGGGCAGCTGGTCGTGGACGCCCCGGTGCGCCCCCGTAAACTACTAGGTAAATCTCACTACGCTCGGGCAACGGAAAACTCAACCAGAACGGACGAGCAAATC',
'TTCAGCAGACATGAGTACTATTTAGCTGTCTACAGAAGCTGGTCCTCCACGGACTGTCTGATTGCTAGGGGCCCAAATGATTACTTATATCGTACACATATTGCAGATTTGAAA',
'CGCACCGCATAGTCACAGACGGGCCTTGTCGTCCTCGGGCAGGAGCGTATCTCGACAGTGGCCGCTACCATCGTACATGGTCCAGAGTAGAATAGGAATCACTCGTGTGAACAT',]


In [ ]:
print(DNA_sequences)

['GTATCTACTCACGTTGCTACCTGCGGTTCCGATTAGGATCAGGTTATAACGTAGTCGCCCTGATATGACTACATCGTTGCAGGCTTGTTACACGTACTAACATATTGGAAAACG', 'TGCCCACTCCCTGATACGCGTTGACCGATCGTGGTGATTTCTTGATCTAAGTCGCCACCCTCTTCAAGGGCTATACATCAACAGAAATTTGTCTGTTTTAGGTCTGTGCCCGCC', 'CTAGACGTAGCGATATGTCGTATGGGACGGGATGTCGCGTGCAAAAATGACCCCTCGGAACCGGCTAAGGCAGCTGTCTATGAGTGGTATTCCTGCTAGGCGTTAACTCGGACT', 'GAGGACCCGGGTACTTGCAGCAAGGCGTGAAAGAAGAAATAATGGTCTATAAAGTGCCCCTGATTCCTGCTCTCAAGGCACCTCGGTGGGGTGCGTTAGGCATAATTTGTTTGT', 'GCCAACACATCCGACGAGAGGTATCAAGCGTACCGCACGTTCACAGGAGATTCCATGGCGACTTAAGTAGATCGTCGTGCACAAAGGCGCCAGACCGATGATACGCGTACAATC', 'ATTCGTCGACGACACCTCCTGCTCATAGTCGCGACCTTAACCTTCTCCACCTTGACTACCCCGGCCTGAGCAAACGCTTCATTAAACATAGAAATTAACCAAGACGGTGGATAG', 'CGCGGCTCTAACTCAAACCCGACAACTCCGAGGATCCCGGGTGAGCATTATCGTGGGCAATTTTACTGCCGGGTTCTGGTAGTAGTCGCGCAGGTAACGCGCGAAGCTTGTGTC', 'TCAATGACAACGGGGCAGCTGGTCGTGGACGCCCCGGTGCGCCCCCGTAAACTACTAGGTAAATCTCACTACGCTCGGGCAACGGAAAACTCAACCAGAACGGACGAGCAAATC', 'TTCAGCAGACATGAGTACTATTTAGCTGTCTACAGAAGCTGGTCCTCCACGGAC

**Greedy Motif Search: Computation**

In [ ]:
k = 15
best_motifs = GreedyMotifSearch(DNA_sequences, k)

print('The best motifs found:')
for i, s in enumerate(DNA_sequences):
    print(f'{str(i).rjust(2)}, motif location: {DNA_sequences[i].find(best_motifs[i])}, motif: {best_motifs[i]}')

Greedy search for motifs of length 15. 
The best solution that is found is returned.
The best motifs found:
 0, motif location: 83, motif: CTTGTTACACGTACT
 1, motif location: 0, motif: TGCCCACTCCCTGAT
 2, motif location: 0, motif: CTAGACGTAGCGATA
 3, motif location: 0, motif: GAGGACCCGGGTACT
 4, motif location: 96, motif: GATGATACGCGTACA
 5, motif location: 0, motif: ATTCGTCGACGACAC
 6, motif location: 83, motif: AGTCGCGCAGGTAAC
 7, motif location: 36, motif: GTGCGCCCCCGTAAA
 8, motif location: 40, motif: GGTCCTCCACGGACT
 9, motif location: 10, motif: AGTCACAGACGGGCC
